### 1: Dependencies

In [2]:
# Langchain dependencies
from langchain.document_loaders.pdf import PyPDFDirectoryLoader  # Importing PDF loader from Langchain
from langchain.document_loaders import CSVLoader  # Importing CSV loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Importing text splitter from Langchain
from langchain.embeddings import OpenAIEmbeddings  # Importing OpenAI embeddings from Langchain
from langchain.schema import Document  # Importing Document schema from Langchain
from langchain.vectorstores.chroma import Chroma  # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI


import os  # Importing os module for operating system functionalities
import shutil  # Importing shutil module for high-level file operations

### 2: Read PDF

In [7]:
# Directory to your pdf files:
DATA_PATH = r"docs"

def load_documents():
    """
    Load PDF documents from the specified directory using PyPDFDirectoryLoader.

    Returns:
        List of Document objects: Loaded PDF documents represented as Langchain Document objects.
    """
    document_loader = CSVLoader(DATA_PATH + "/test_data.csv")  # Initialize PDF loader with specified directory
    return document_loader.load()  # Load PDF documents and return them as a list of Document objects

In [8]:
documents = load_documents()
print(documents)

[Document(metadata={'source': 'docs/test_data.csv', 'row': 0}, page_content='\ufefftitle: 팬텀 버스터즈 2 - S코믹스\ndescription: 멤버를 모아,  팬텀 버스터즈로서 제령 활동을 시작한 모가리와 친구들. 인지도를 높이기 위한 홍보 활동을 하던 중, 냉철한 현실주의자로 알려진 이치미야 학생회장에게 찍히고(?) 말았다! 게다가 시시쿠노 영산에서는 모가리 암살이라는 사명을 받은 자객이 파견되는데?!\nKAKAO_contents: 멤버를 모아, 팬텀 버스터즈로서 제령 활동을 시작한 모가리와 친구들. 인지도를 높이기 위한 홍보 활동을 하던 중, 냉철한 현실주의자로 알려진 이치미야 학생회장에게 찍히고(?) 말았다! 게다가 시시쿠노 영산에서는 모가리 암살이라는 사명을 받은 자객이 파견되는데--?!  4명의 남고생들이 펼치는 우당탕탕 제령×청춘 코미디!\nNaver_description: SNS에서도 화제만발!\n본격적으로 활동을 시작한 팬버스의\n앞을 가로막는 자객과 시련!!!\n\n멤버를 모아,  팬텀 버스터즈로서\n제령 활동을 시작한 모가리와 친구들. \n인지도를 높이기 위한 홍보 활동을 하던 중, \n냉철한 현실주의자로 알려진 \n이치미야 학생회장에게 찍히고(?) 말았다!\n게다가 시시쿠노 영산에서는 모가리 암살이라는\n사명을 받은 자객이 파견되는데--?!\n\n4명의 남고생들이 펼치는\n우당탕탕 제령×청춘 코미디!'), Document(metadata={'source': 'docs/test_data.csv', 'row': 1}, page_content="\ufefftitle: 소년이 온다 - 2024 노벨문학상 수상작가\ndescription: 섬세한 감수성과 치밀한 문장으로 인간 존재의 본질을 탐구해온 작가 한강의 여섯번째 장편소설. '상처의 구조에 대한 투시와 천착의 서사'를 통해 한강만이 풀어낼 수 있는 방식으로 1980년 5월을 새롭게 조명한다.\nKAKAO_contents:

In [9]:
type(documents)

list

### 3: Split into chunks of text

Is this step necessary or useful for my application?

In [10]:
def split_text(documents: list[Document]):
    """
    Split the text content of the given list of Document objects into smaller chunks.

    Args:
        documents (list[Document]): List of Document objects containing text content to split.

    Returns:
        list[Document]: List of Document objects representing the split text chunks.
    """
    # Initialize text splitter with specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,  # Size of each chunk in characters
        chunk_overlap=100,  # Overlap between consecutive chunks
        length_function=len,  # Function to compute the length of the text
        add_start_index=True,  # Flag to add start index to each chunk
    )
    # Split documents into smaller chunks using text splitter
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    # Print example of page content and metadata for a chunk
    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks  # Return the list of split text chunks

In [11]:
chunks = split_text(documents)

Split 10 documents into 36 chunks.
2016년 『채식주의자』로 인터내셔널 부커상을 수상하고 2018년 『흰』으로 같은 상 최종 후보에 오른 한강 작가의 5년 만의 신작 장편소설 『작별하지 않는다』가 출간되었다. 2019년 겨울부터 이듬해 봄까지 계간 『문학동네』에 전반부를 연재하면서부터 큰 관심을 모았고, 그뒤 일 년여에 걸쳐 후반부를 집필하고 또 전체를 공들여 다듬는 지난한 과정을 거쳐 완성되었다. 본래 「눈 한 송이가 녹는 동안」(2015년 황순원문학상 수상작), 「작별」(2018년 김유정문학상 수상작)을 잇는 ‘눈’ 3부작의 마지막 작품으로 구상되었으나 그 자체 완결된 작품의 형태로 엮이게 된바, 한강 작가의 문학적 궤적에서 『작별하지 않는다』가 지니는 각별한 의미를 짚어볼 수 있다. 이로써 『소년이 온다』(2014),
{'source': 'docs/test_data.csv', 'row': 2, 'start_index': 607}


In [13]:
for chunk in chunks:
    print(chunk)
    print("\n")

page_content='﻿title: 팬텀 버스터즈 2 - S코믹스
description: 멤버를 모아,  팬텀 버스터즈로서 제령 활동을 시작한 모가리와 친구들. 인지도를 높이기 위한 홍보 활동을 하던 중, 냉철한 현실주의자로 알려진 이치미야 학생회장에게 찍히고(?) 말았다! 게다가 시시쿠노 영산에서는 모가리 암살이라는 사명을 받은 자객이 파견되는데?!
KAKAO_contents: 멤버를 모아, 팬텀 버스터즈로서 제령 활동을 시작한 모가리와 친구들. 인지도를 높이기 위한 홍보 활동을 하던 중, 냉철한 현실주의자로 알려진 이치미야 학생회장에게 찍히고(?) 말았다! 게다가 시시쿠노 영산에서는 모가리 암살이라는 사명을 받은 자객이 파견되는데--?!  4명의 남고생들이 펼치는 우당탕탕 제령×청춘 코미디!' metadata={'source': 'docs/test_data.csv', 'row': 0, 'start_index': 0}


page_content='Naver_description: SNS에서도 화제만발!
본격적으로 활동을 시작한 팬버스의
앞을 가로막는 자객과 시련!!!' metadata={'source': 'docs/test_data.csv', 'row': 0, 'start_index': 382}


page_content='멤버를 모아,  팬텀 버스터즈로서
제령 활동을 시작한 모가리와 친구들. 
인지도를 높이기 위한 홍보 활동을 하던 중, 
냉철한 현실주의자로 알려진 
이치미야 학생회장에게 찍히고(?) 말았다!
게다가 시시쿠노 영산에서는 모가리 암살이라는
사명을 받은 자객이 파견되는데--?!

4명의 남고생들이 펼치는
우당탕탕 제령×청춘 코미디!' metadata={'source': 'docs/test_data.csv', 'row': 0, 'start_index': 452}


page_content='﻿title: 소년이 온다 - 2024 노벨문학상 수상작가
description: 섬세한 감수성과 치밀한 문장으로 인간 존재의 본질을 탐구해온 작가 한강의

### 4: Save to a RDB using Chroma

In [14]:
CHROMA_PATH = "chroma"

In [15]:
def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
    
    # print(chunks)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

### 5: Create a Chroma Database

In [16]:
def generate_data_store():
    """
    Function to generate vector database in chroma from documents.
    """
    documents = load_documents()  # Load documents from a source
    chunks = split_text(documents)  # Split documents into manageable chunks
    save_to_chroma(chunks)  # Save the processed data to a data store


In [17]:
# Load environment variables from a .env file
load_dotenv()
# Generate the data store
generate_data_store()

Split 10 documents into 36 chunks.
2016년 『채식주의자』로 인터내셔널 부커상을 수상하고 2018년 『흰』으로 같은 상 최종 후보에 오른 한강 작가의 5년 만의 신작 장편소설 『작별하지 않는다』가 출간되었다. 2019년 겨울부터 이듬해 봄까지 계간 『문학동네』에 전반부를 연재하면서부터 큰 관심을 모았고, 그뒤 일 년여에 걸쳐 후반부를 집필하고 또 전체를 공들여 다듬는 지난한 과정을 거쳐 완성되었다. 본래 「눈 한 송이가 녹는 동안」(2015년 황순원문학상 수상작), 「작별」(2018년 김유정문학상 수상작)을 잇는 ‘눈’ 3부작의 마지막 작품으로 구상되었으나 그 자체 완결된 작품의 형태로 엮이게 된바, 한강 작가의 문학적 궤적에서 『작별하지 않는다』가 지니는 각별한 의미를 짚어볼 수 있다. 이로써 『소년이 온다』(2014),
{'source': 'docs/test_data.csv', 'row': 2, 'start_index': 607}


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_10628\2280540442.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH


Saved 36 chunks to chroma.


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_10628\2280540442.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


#### Embedding example

In [18]:
ex = "apple"
ex_1 = "orange"
ex_2 = "iphone"

In [19]:
embedding_function = OpenAIEmbeddings()
vector = embedding_function.embed_query(ex)
vector_1 = embedding_function.embed_query(ex_1)
vector_2 = embedding_function.embed_query(ex_2)

In [20]:
vector, len(vector)

([0.007730894059324105,
  -0.02313804706855683,
  -0.007587476431447153,
  -0.027809365687387712,
  -0.004650829521477468,
  0.013010029201724864,
  -0.02196338849083348,
  -0.008393346623632073,
  0.018958446297110524,
  -0.02955769477661887,
  -0.0029264030597849337,
  0.020078469765418393,
  -0.004415214680800583,
  0.009158240949416682,
  -0.021649234749049225,
  0.002014676411784905,
  0.030732353354342223,
  0.00010212104452982253,
  0.0020266278419695427,
  -0.025460046669295792,
  -0.02106190546018755,
  -0.008195294352000941,
  0.0213760592019718,
  -0.012552459229047571,
  0.001133682362077065,
  0.005043520767385176,
  0.010196311753601323,
  7.81647495259728e-05,
  0.016062776184863757,
  -0.013023687979078734,
  0.020460917393972003,
  -0.016158387626340857,
  -0.01838477578560272,
  0.00544304140196982,
  -0.019381870257725746,
  -0.009171899726770554,
  -0.01203342382695525,
  -0.008707500365416326,
  -0.005702558637354677,
  -0.006166958930031512,
  0.010524123341416839

In [21]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("pairwise_embedding_distance")

In [22]:
# run an evaluation

x = evaluator.evaluate_string_pairs(prediction=ex, prediction_b=ex_1)

In [23]:
x

{'score': 0.13554126333631622}

In [24]:
evaluator.evaluate_string_pairs(prediction=ex, prediction_b=ex_2)

{'score': 0.09709082173706307}

In [25]:
evaluator.evaluate_string_pairs(prediction=ex, prediction_b=ex)

{'score': -2.220446049250313e-16}

Bigger distance = strings are more different

### 6: Query vector database for relevant data

In [32]:
import pandas as pd

df = pd.read_csv("docs/test_data.csv")
titles = list(df['title'].values)

In [26]:
query_text = titles[1] + "이 소설에 대한 쟁점"
print(titles[1])

In [34]:
PROMPT_TEMPLATE = """
{question}의 첫 단락은 책 제목입니다.

{context}를 기반으로 주어진 책과 관련된 질문에 답하십시오.
오직 주어진 문맥만 사용하여 답변하십시오.

---

Answer the question based on the above context: {question}
"""


In [35]:
# Use same embedding function as before
embedding_function = OpenAIEmbeddings()
 
# Prepare the database
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=3)
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_10628\1276036629.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)


In [36]:
from langchain.prompts import ChatPromptTemplate

In [37]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)

Human: 
이 소설에 대한 쟁점의 첫 단락은 책 제목입니다.

2016년 인터내셔널 부커상을 수상하며 한국문학의 입지를 한단계 확장시킨 한강의 장편소설 『채식주의자』를 15년 만에 새로운 장정으로 선보인다. 상처받은 영혼의 고통과 식물적 상상력의 강렬한 결합을 정교한 구성과 흡인력 있는 문체로 보여주는 이 작품은 섬뜩한 아름다움의 미학을 한강만의 방식으로 완성한 역작이다. “탄탄하고 정교하며 충격적인 작품으로, 독자들의 마음에 그리고 아마도 그들의 꿈에 오래도록 머물 것이다”라는 평을 받으며 인터내셔널 부커상을 수상했던 『채식주의자』는 “미국 문학계에 파문을 일으키면서도 독자들과 공명할 것으로 보인다”(뉴욕타임스), “놀라울 정도로 아름다운 산문과 믿을 수 없을 만큼 폭력적인 내용의 조합이 충격적이다”(가디언)라는 해외서평을 받았고 2018년에는 스페인에서 산클레멘테

---

『소년이 온다』는 ‘상처의 구조에 대한 투시와 천착의 서사’를 통해 한강만이 풀어낼 수 있는 방식으로 1980년 5월을 새롭게 조명하며, 무고한 영혼들의 말을 대신 전하는 듯한 진심 어린 문장들로 5·18 이후를 살고 있는 우리에게 묵직한 질문을 던진다. 
이 작품은 가장 한국적인 서사로 세계를 사로잡은 한강 문학의 지향점을 보여준다. 인간의 잔혹함과 위대함을 동시에 증언하는 이 충일한 서사는 이렇듯 시공간의 한계를 넘어 인간 역사의 보편성을 보여주며 훼손되지 말아야 할 인간성을 절박하게 복원한다.

---

평을 보내며 이 소설이 키건의 정수가 담긴 작품임을 알렸다. 전 세계 독자들의 사랑과 언론의 호평을 받으며 베스트셀러에 오른 이 책은, 자신이 속한 사회 공동체의 은밀한 공모를 발견하고 자칫 모든 걸 잃을 수 있는 선택 앞에서 고뇌하는 한 남자의 내면을 그린 작품이다. 키건 특유의 섬세한 관찰과 정교한 문체로 한 인간의 도덕적 동요와 내적 갈등, 실존적 고민을 치밀하게 담아냈다. 저자의 열렬한 팬으로 유명한 아일랜드 출신의 배우 킬리언 머피는 직접 제작과 주연을 맡아 이 소설을

In [25]:
model = ChatOpenAI()
response_text = model.predict(prompt)

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

c:\Users\callu\repos\lessons\1. RAG\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\callu\repos\lessons\1. RAG\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response: The YOLO method works by dividing the image into a grid of S x S cells, where each cell is responsible for predicting up to two objects from a single class. The model uses a regression-based approach to predict the bounding boxes for these objects. However, this design constraint of only being able to predict two objects per cell makes it difficult for the model to accurately detect multiple small objects that are close to each other in the image space. This limitation is highlighted by the struggle of the model to detect an entire flock of birds.
Sources: ['data\\MACPHERSON, Callum - MSc Dissertation.pdf', 'data\\MACPHERSON, Callum - MSc Dissertation.pdf', 'data\\MACPHERSON, Callum - MSc Dissertation.pdf']


In [26]:
response_text

'The YOLO method works by dividing the image into a grid of S x S cells, where each cell is responsible for predicting up to two objects from a single class. The model uses a regression-based approach to predict the bounding boxes for these objects. However, this design constraint of only being able to predict two objects per cell makes it difficult for the model to accurately detect multiple small objects that are close to each other in the image space. This limitation is highlighted by the struggle of the model to detect an entire flock of birds.'

In [27]:
def query_rag(query_text):
    """
    Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.

    Args:
    - query_text (str): The text to query the RAG system with.

    Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
    """
    # Use same embedding function as before
    embedding_function = OpenAIEmbeddings()
    
    # Prepare the database
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    
    # Check if there are any matching results or if the relevance score is too low
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")

    # Combine context from matching documents
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    
    # Create prompt template using context and query text
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    # Initialize OpenAI chat model
    model = ChatOpenAI()
    
    # Generate response text based on the prompt
    response_text = model.predict(prompt)

    # Get sources of the matching documents
    sources = [doc.metadata.get("source", None) for doc, _score in results]
    
    # Format and return response including generated text and sources
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    return formatted_response, response_text

In [28]:
formatted_response, response_text = query_rag(query_text)

In [29]:
response_text

'The YOLO method works by splitting the image into a grid and using a regression-based approach to predict bounding boxes for objects within each grid cell. Each cell can only predict up to two objects from one class, which can pose spatial constraints and make it difficult to detect multiple small objects in close proximity. This design choice can make it challenging for the model to accurately detect entire groups of objects, such as a flock of birds, in the image space.'